In [2]:
# edit date : 2024-03-05
# version : 1.9.0

from selenium import webdriver
import webbrowser
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import *

import time
import datetime as dt

import random
from random import randint

import os
import win32api

from PyKakao import Message
import requests
import webbrowser
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

########################################################################################################
## Chrome Driver 불러오기
def get_driverByPath(pathDriver):
    # pathDriver = r"C:\Users\hanta\.wdm\drivers\chromedriver\win64"
    subfolders = [f.name for f in os.scandir(pathDriver) if f.is_dir()]
    downloaded_latest_driver_version = max(subfolders)
    
    try:
        driver_path = rf"{pathDriver}\{downloaded_latest_driver_version}\chromedriver-win32\chromedriver.exe"
        driver = webdriver.Chrome(service=ChromeService(executable_path=driver_path))
        return driver
    except:
        driver_path = ChromeDriverManager().install()
        driver = webdriver.Chrome(service=ChromeService(executable_path=driver_path))
        return driver

    
########################################################################################################
## Chrome Driver 실행
def ExcutionbyDriver(input_driver):
    
    # Reservation info.
    #=============================================================================
    member_number = "2092236381" # 회원번호
    password= "thfldjqtdl2@" # 비밀번호
    arrival = "대전" # 출발지
    departure = "수서" # 도착지
    standard_date = "20241122" # 기준날짜 ex) 20221101
    standard_time = "16" # 기준 시간 ex) 00 - 22 // 2의 배수로 입력
    #=============================================================================
    
    driver = input_driver
    driver.get('https://etk.srail.co.kr/cmc/01/selectLoginForm.do')
    driver.implicitly_wait(15)

    # 회원번호 매핑
    driver.find_element(By.ID, 'srchDvNm01').send_keys(member_number)

    # 비밀번호 매핑
    driver.find_element(By.ID, 'hmpgPwdCphd01').send_keys(password)

    # 확인 버튼 클릭
    driver.find_element(By.XPATH, '/html/body/div/div[4]/div/div[2]/form/\
        fieldset/div[1]/div[1]/div[2]/div/div[2]/input').click()
    driver.implicitly_wait(5)

    driver.get('https://etk.srail.kr/hpg/hra/01/selectScheduleList.do')
    driver.implicitly_wait(5)


    # 출발지 입력
    dep_stn = driver.find_element(By.ID, 'dptRsStnCdNm')
    dep_stn.clear()
    dep_stn.send_keys(arrival)

    # 도착지 입력
    arr_stn = driver.find_element(By.ID, 'arvRsStnCdNm')
    arr_stn.clear()
    arr_stn.send_keys(departure)

    # 날짜 드롭다운 리스트 보이게
    # elm_dptDt = driver.find_element(By.ID, "dptDt")
    # driver.execute_script("arguments[0].setAttribute('style','display: True;)", elm_dptDt)
    Select(driver.find_element(By.ID,"dptDt")).select_by_value(standard_date)

    # 출발 시간
    # eml_dptTm = driver.find_element(By.ID, "dptTm")
    # driver.execute_script("arguments[0].setAttribbute('style','display:True;')", eml_dptTm)
    Select(driver.find_element(By.ID, "dptTm")).select_by_visible_text(standard_time)

    # 조회하기 버튼
    driver.find_element(By.XPATH, "//input[@value='조회하기']").click()
    train_list = driver.find_elements(By.CSS_SELECTOR, "#result-form > fieldset > \
    div.tbl_wrap.th_thead > table > tbody > tr")    

    
    
########################################################################################################
def now():
    return dt.datetime.now()
    
    
########################################################################################################
## kakao Message API Function
def kakaoMessage(type_contents=None, count=None):
    rest_api = 'c7331f1eedd34ad746c501eb91d59e1f'
    redirect_url = 'https://localhost:5000'
    token_url = 'https://kauth.kakao.com/oauth/token'
    
    # default Info
    init_token = '2E424TAMSgy692PWWxz3QEJ7MJmumDNoAAAAAQoqJVAAAAGTGcYybNEMsmlHt4Ko'
    refresh_token = 'g71d09rp9sxQCevC_bTy5f15WEUk7RuuAAAAAgoqJVAAAAGTGcYyaNEMsmlHt4Ko'

    # token 만료 확인
    token_info_url = "https://kapi.kakao.com/v1/user/access_token_info"
    token_info_headers = {
            "Authorization": "Bearer " + init_token
        }
    toke_info_response = requests.get(token_info_url, headers=token_info_headers)
    
    # 아직 유효
    if toke_info_response.status_code==200:
        valid_access_token = init_token
    else:
        re_data = {
            "grant_type" : "refresh_token",
            "client_id" : rest_api,
            "refresh_token" : refresh_token
        }
        re_response = requests.post(token_url,data=re_data)
        re_token = re_response.json()
        valid_access_token = re_token['access_token']

    api = Message(service_key  = rest_api)
    api.set_access_token(valid_access_token)
    message_type = "text"
    
    # UUID 목록
    receiver_uuids = [
        "NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ", 
    ]

    if type_contents:
        # 파라미터
        text = f"SRT 등록 완료 by 남기원 {count}"
        link = {
                    "web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000",
                    "mobile_web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000"
                }
        button_title = "바로 확인"
    else:
        text = f"매크로 체크용 : No.{count}"
        link = {
                    "web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000",
                    "mobile_web_url": "https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000"
                }
        button_title = "바로 확인"
    
    api.send_message_to_friend(
        message_type=message_type, 
        receiver_uuids=receiver_uuids,
        text=text,
        link=link,
        button_title=button_title,
    )

    
########################################################################################################
## SRT reserving Function
def Repeat_reserved(start_of_trains, number_of_trains, input_driver, message_count):
    chrome_path = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s'
    reserved = False
    count = 0
    driver = input_driver
    while True: 
        count +=1
        try:
            driver.implicitly_wait(300)
            for i in range(start_of_trains, number_of_trains):
                # wait = WebDriverWait(driver, 300)
                standard_seat = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(7)").text
                reserved_seat = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(8)").text
                if "예약하기" in standard_seat:
                    print(f"예약 가능 클릭 : {now()}", end='\n')
                    driver.find_element(By.XPATH, f"/html/body/div[1]/div[4]/div/div[3]/div[1]/form/fieldset/div[6]/table/tbody/tr[{i}]/td[7]/a/span").click()
                    driver.implicitly_wait(3)
                    if driver.find_elements(By.ID, 'isFalseGotoMain'):
                        reserved = True
                        print(f'예약 성공 : {now()}', end='\n')
                        webbrowser.get(chrome_path).open("https://etk.srail.kr/hpg/hra/02/selectReservationList.do?pageId=TK0102010000")
                        win32api.MessageBox(0, "Seat Reserved", "Done", 64)
                        kakaoMessage(type_contents=True, count=count)
                        break

                    else:
                        driver.back() #뒤로가기
                        print(f"잔여석 없음. 다시 검색 : {now()}", end='\n')
                        driver.implicitly_wait(5)
 

        except: 
            print('잔여석 조회 불가', end='\n')
            pass

        if not reserved:
            try:
            # 다시 조회하기
                # wait = WebDriverWait(driver, 300)
                # submit_element = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/div[4]/div/div[2]/form/fieldset/div[2]/input")))
                driver.implicitly_wait(300)
                submit = driver.find_element(By.XPATH, "/html/body/div/div[4]/div/div[2]/form/fieldset/div[2]/input")
                driver.execute_script("arguments[0].click();", submit)                          
                print(f"새로고침{count}", end='\r')
                if count%message_count==0:
                    kakaoMessage(type_contents=False, count=count)
                    
            except: 
                print(f"잔여석 없음 #2. 초기화 : {now()}", end='\n')
                driver.back() #뒤로가기
                driver.implicitly_wait(10)

                driver.refresh() #새로고침
                driver.implicitly_wait(10)
                pass

            # 대기
            driver.implicitly_wait(300)
            randomndelay=random.randrange(3000,6001)/1000
            time.sleep(randomndelay)

        else:
            print("오류", end='\n')
            time.sleep(1000)
            break

In [ ]:
## Desktop, Path Setting
import socket

current_desktop = socket.gethostname()
print(f"Desktop Name : {current_desktop}")

# 폴더 경로 지정
if 'OCL' in current_desktop :
    folder_path = r"C:\Users\hanta\Desktop\study\python/local-home-4090"
    pathDriver = r"C:\Users\hanta\.wdm\drivers\chromedriver\win64"
else :
    folder_path = r"C:\Users\Administrator\Desktop\code/local-home-4090"
    pathDriver = r"C:\Users\Administrator\.wdm\drivers\chromedriver\win64"

print(f"Folder_path : {folder_path} \nDriver_path : {pathDriver}")

driver = get_driverByPath(pathDriver)
ExcutionbyDriver(driver)

start_of_trains = 3 # 검색 시 보여지는 기차 중 시작할 기차의 서수
number_of_trains = 3 # 상단에서부터 조회할 기차수  maximum = 11
message_count = 100

Repeat_reserved(start_of_trains,number_of_trains, driver, message_count)

Desktop Name : OCL2208007
Folder_path : C:\Users\hanta\Desktop\study\python/local-home-4090 
Driver_path : C:\Users\hanta\.wdm\drivers\chromedriver\win64
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']
액세스 토큰 설정 완료
메시지 전송 성공: ['NwM0ADkBMgQ1GSoYLxYuGCEYNAU1AzsMNQFJ']


In [47]:
kakaoMessage(1)

PvsI0DTSB8frlZIqOWQh6_xYByr5dMDGHw8KPXSZAAABjyxib-HRDLJpR7eCqA
액세스 토큰 설정 완료
메시지 전송 성공


In [ ]:
## Github Setting
# git remote 명령어 실행
# import subprocess
# remote_url = "https://github.com/your_username/your_repository.git"
# subprocess.run(["git", "-C", folder_path, "remote", "add", "origin", remote_url])

# git pull 명령어 실행
# subprocess.run(["git", "-C", folder_path, "pull"])